In [43]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [44]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [45]:
# Loading the training and testing data into dataframes
dir_path = "/kaggle/input/nlp-getting-started/"
train = pd.read_csv(dir_path + "train.csv")
test= pd.read_csv(dir_path + "test.csv")

In [46]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [47]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [48]:
train_df=train[['text','target']]

train_df.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [49]:
import re
def process_tweet(tweet):
    return " ".join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])", "",tweet.lower()).split())

In [50]:
processed_text=[]
for item in range(train_df.shape[0]):
     processed_text.append(process_tweet(train_df['text'][item]))

In [51]:
train_df['processed_text']=processed_text
train_df.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,text,target,processed_text
0,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...
1,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...
3,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...
4,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...


In [52]:
print(train_df['text'][3])
print(train_df['processed_text'][3])

13,000 people receive #wildfires evacuation orders in California 
13000 people receive wildfires evacuation orders in california


In [53]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(train_df['processed_text'].values)
X = tokenizer.texts_to_sequences(train_df['processed_text'].values)
X = pad_sequences(X)

In [54]:
train_df.tail()

,text,target,processed_text
7608,Two giant cranes holding a bridge collapse int...,1,two giant cranes holding a bridge collapse int...
7609,@aria_ahrary @TheTawniest The out of control w...,1,ahrary the out of control wild fires in califo...
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,m194 0104 utc5km s of volcano hawaii httptcozd...
7611,Police investigating after an e-bike collided ...,1,police investigating after an ebike collided w...
7612,The Latest: More Homes Razed by Northern Calif...,1,the latest more homes razed by northern califo...


In [55]:
print(train_df.shape)

(7613, 3)


In [56]:
y=pd.get_dummies(train_df['target']).values
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=23)

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(6851, 30) (6851, 2)
(762, 30) (762, 2)


In [57]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(n_estimators=200)

model.fit(X_train,y_train)

RandomForestClassifier(n_estimators=200)

In [58]:
y_pred=np.argmax(model.predict(X_test), axis=-1)

Y_test=np.argmax(y_test,axis=-1)

print(y_pred)
print(Y_test)

[0 0 0 0 1 0 0 0 0 1 0 0 1 1 1 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0
 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 1 1 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 0
 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0
 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 0 1 0 0 1 1 0 1 1 0 0 0 1 0
 0 0 0 0 0 1 0 1 0 0 0 1 1 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0
 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 1 0 0 0 1 1 0 0 1 0
 0 1 0 0 0 0 1 0 1 1 0 0 0 0 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 1
 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 1 1 0 1
 0 0 0 1 0 1 0 1 1 1 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0
 0 1 0 1 0 1 0 0 1 0 0 1 1 1 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0 1
 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 1
 0 1 0 1 1 1 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1
 0 0 0 0 0 0 1 1 0 0 0 0 

In [59]:
from sklearn.metrics import classification_report as cr

print(cr(Y_test,y_pred))

              precision    recall  f1-score   support

           0       0.67      0.83      0.74       432
           1       0.68      0.46      0.55       330

    accuracy                           0.67       762
   macro avg       0.67      0.65      0.65       762
weighted avg       0.67      0.67      0.66       762



In [60]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [61]:
test=test[['text']]

print(test.tail())

print(test.shape)


                                                   text
3258  EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259  Storm in RI worse than last hurricane. My city...
3260  Green Line derailment in Chicago http://t.co/U...
3261  MEG issues Hazardous Weather Outlook (HWO) htt...
3262  #CityofCalgary has activated its Municipal Eme...
(3263, 1)


In [62]:
test_processed_text=[]
for item in range(test.shape[0]):
     test_processed_text.append(process_tweet(test['text'][item]))

In [63]:
test['processed_text']=test_processed_text

test.tail()

,text,processed_text
3258,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,earthquake safety los angeles safety fasteners...
3259,Storm in RI worse than last hurricane. My city...,storm in ri worse than last hurricane my citya...
3260,Green Line derailment in Chicago http://t.co/U...,green line derailment in chicago httptcoutbxlc...
3261,MEG issues Hazardous Weather Outlook (HWO) htt...,meg issues hazardous weather outlook hwo httpt...
3262,#CityofCalgary has activated its Municipal Eme...,cityofcalgary has activated its municipal emer...


In [64]:
print(test.shape)

(3263, 2)


In [69]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(test['processed_text'].values)
test_final = tokenizer.texts_to_sequences(test['processed_text'].values)
test_final = pad_sequences(test_final,maxlen=30)

In [70]:
test_final.shape

(3263, 30)

In [73]:
y_pred_final=np.argmax(model.predict(test_final), axis=-1)

y_pred_final

array([0, 1, 0, ..., 1, 0, 0])

In [74]:
sample=pd.read_csv(dir_path + "test.csv")
sample['target']=y_pred_final

print(sample.head())

print(sample.shape)

   id keyword location                                               text  \
0   0     NaN      NaN                 Just happened a terrible car crash   
1   2     NaN      NaN  Heard about #earthquake is different cities, s...   
2   3     NaN      NaN  there is a forest fire at spot pond, geese are...   
3   9     NaN      NaN           Apocalypse lighting. #Spokane #wildfires   
4  11     NaN      NaN      Typhoon Soudelor kills 28 in China and Taiwan   

   target  
0       0  
1       1  
2       0  
3       0  
4       1  
(3263, 5)


In [75]:
sample.head()


,id,keyword,location,text,target
0,0,NaN,NaN,Just happened a terrible car crash,0
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",1
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",0
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,0
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,1


In [76]:
sample=sample[['text','target']]

sample.head()

,text,target
0,Just happened a terrible car crash,0
1,"Heard about #earthquake is different cities, s...",1
2,"there is a forest fire at spot pond, geese are...",0
3,Apocalypse lighting. #Spokane #wildfires,0
4,Typhoon Soudelor kills 28 in China and Taiwan,1


In [77]:
sample.to_csv('submission.csv',index=False)